# Predictor de diabetes

## Objetivo

El objetivo de este proyecto es conseguir un modelo que brinde la probabilidad de que de un paciente tenga diabetes a partir de ciertas medidas de diagnóstico clínico.

### Análisis exploratorio y visualización efectiva

Antes de realizar un modelo que prediga la probabilidad de padecer diabetes, se exploró el [dataset](https://www.kaggle.com/datasets/whenamancodes/predict-diabities?resource=download) para evaluar su integridad y generar algunas visualizaciones que pueda brindar información a priori sobre los datos de diagnóstico y el poseer o no diabetes.
Algunas de las relaciones que se estudiaron fueron: 

1. ¿Qué relación hay entre la glucemia y la insulina con la aparición de diabetes en los pacientes?
2. ¿Qué relación hay entre el índice de masa corporal (BMI) con la diabetes?
3. ¿El número de embarazos en las pacientes influye en la aparición de diabetes?


## Fuente de datos

El dataset se descargó de [Kaggle](https://www.kaggle.com/datasets/whenamancodes/predict-diabities?resource=download) y posee de 9 columnas y 768 filas: 

| Columnas                 | Descripción                                      |
| ------------------------ | ------------------------------------------------ |
| Pregnancies              | To express the Number of pregnancies             |
| Glucose                  | To express the Glucose level in blood            |
| BloodPressure            | To express the Blood pressure measurement        |
| SkinThickness            | To express the thickness of the skin             |
| Insulin                  | To express the Insulin level in blood            |
| BMI                      | To express the Body mass index                   |
| DiabetesPedigreeFunction | To express the Diabetes percentage               |
| Age                      | To express the age                               |
| Outcome                  | To express the final result 1 is Yes and 0 is No |

Este conjunto de datos es originalmente del National Institute of Diabetes and Digestive and Kidney Diseases.
Para la generación de este dataset se colocaron varias restricciones en la selección de los datos, en particular, este dataset todos los pacientes son mujeres de al menos 21 años de edad de ascendencia indígena Pima.

## Análisis exploratorio de los datos y visualización efectiva

El dataset utilizado no presenta filas repetidas o valores faltantes, pero algunas columnas como Glucose, BloodPressure, SkinThickness, Insulin y BMI presentan valores mínimos iguales a cero. 
La cantidad de filas con este valor en cada una de las columnas es: 

- Glucose : 5 (0 %)
- BloodPressure : 35 (4 %)
- SkinThickness : 227 (29 %)
- Insulin : 374 (48 %)
- BMI : 11 (1 %)

La cantidad de filas comprometidas es muy significativo en algunas de las columnas como BloodPressure, SkinThickness y Insulin, por lo cual no es aconsejable eliminarlas directamente.

Por ello se tomó la decisión de remplazar esos valores teniendo en cuenta la siguiente fórmula:

$$ \mu + (2 . \sigma . np.random.random() - 0.5) $$

Donde para una dada columna, $\mu$ es su valor medio y $\sigma$ su desviación estándar.

Se utilizó esta fórmula, ya que se observó que las columnas de BloodPressure, SkinThickness y Insulin presentan casi una distribución normal como se aprecia en la Fig. 1. 
Además de esta forma no se acumularían valores en un valor en particular, sino que estarían aleatoriamente repartidos con una distribución normal alrededor de $\mu$.

![fig/fig1.png](fig/fig1.png)

Para las columnas de Glucose y BMI se tomó la decisión de eliminar las filas que presentaban valores iguales a cero, ya que no son un gran número de filas y no representan una gran perdida de datos. 
Este dataset con los valores reemplazados se guardó con el nombre de `data/dataset1.csv` y es el que se utilizó para realizar el modelo de predicción.

Además, en la Fig. 1 se puede observar que no hay una clara relación entre la distribución de la edad y el número de embarazos con el diagnóstico de diabetes. 
Así que la respuesta a la pregunta número 3, sería que no hay una clara relación.    

Para poder responder las preguntas 1 y 2 se realizaron las siguientes figuras: Fig. 2, Fig. 3 y Fig.4. 

![fig/fig2.png](fig/fig2.png)

![fig/fig4.png](fig/fig4.png)

La Fig. 2 muestra que los pacientes con niveles de glucosa menores a 80 no tienen posibilidad de tener diabetes.
Mientras que los pacientes con valores entre 80 y 120 comienzan a tener más probabilidad de tener este diagnóstico, especialmente si tienen valores de insulina superiores a 100.
Por otro lado, los pacientes con niveles de glucosa por encima de 120 y altos niveles de insulina, han sido diagnosticados con diabetes.

En la Fig. 3 se aprecia que los pacientes con niveles de glucosa por debajo de 120 y con valores de BMI menores a 30 (peso saludable/normal) rara vez tienen diabetes. 
Al parecer, pacientes con un valor de BMI alto (> 30) y un nivel de glucosa alto (>130) se han diagnosticado con diabetes.

Por último en la Fig. 4  se muestra que los pacientes con un BMI y un Skin Thickness menores a 30 tiene muy baja probabilidad de ser diagnosticados con diabetes. 

### Conclusiones de la visualización de datos

* Un nivel alto de glucosa (> 120) son un indicador de diabetes.

* Los pacientes con BMI alto tienen un mayor riesgo de contraer diabetes.

* El número de embarazos y la eda no seria un determinate para el diagnóstico.

## Modelo de predicción de diabetes

Para realizar el modelo de predicción se utilizó el dataset generado anteriormente (`data/dataser1.csv`) y se usaron 3 diferentes algoritmos de clasificación: 
- Logistisc Regression
- Random Forest y 
- Decision Tree

Para encontrar lo mejores hiperparámetros para cada uno de los diferentes algoritmos se usó la herramienta de `GridSearchCV` y validación cruzada `Kfold` de la librería de `sklearn`.

Al abrir el dataset y graficar la distribución de clases de los datos, como se aprecia en Fig. 5, se observa que las mismas están desbalanceadas, lo cual sugiere que hay más datos de pacientes sin diabetes que con la enfermedad. 
Esto es un tema a tener en cuenta a la hora de realizar los modelos, por lo cual se utilizó a opción `class_weight="balanced"` para balancear al dataset al entrenar los modelos.

![fig/fig5.png](fig/fig5.png)

Los datos del dataset se separaron en 2 grupos: entrenamiento y testeo. 
El primero se utilizó para entrenar el modelo y el segundo para evaluar su performance y solo está formado por el 20% de los datos del dataset.

Antes de realizar esta separación, el dataset se estandarizó, ya que algunas de las variables son más grande que otras.
Para ello se hizo uso de la función `StandardScaler` de `sklearn`.

Luego, para cada uno de los algoritmos se realizó un Grid search para encontrando los respectivos hiperparámetros que maximicen el Accuracy score. 
Posteriormente, se evaluó la performance de cada uno de los algoritmos entrenados con los mejores hiperparámetros obteniendo los scores mostrados en la tabla y las matrices de confusión de la Fig. 6. 

| Modelos         | Logistic regression | Random forest | Decision tree |
| --------------- | ------------------- | ------------- | ------------- |
| Accuracy Score  | 0.78                | 0.84          | 0.8           |
| Precision Score | 0.66                | 0.8           | 0.76          |
| Recall Score    | 0.77                | 0.74          | 0.65          |


Con todos los algoritmos de clasificación se consiguieron scores parecidos, por lo cual se decidió que el mejor sería aquel que minimizara los falsos negativos.
Esta decisión se toma debido a que estos casos son los más riesgo en el ámbito de la medicina, ya que serían paciente que padecen diabetes, pero el modelo predice que no posee diabetes, lo cual es erróneo. 

Teniendo en cuenta este criterio, el mejor modelo se obtiene con el algoritmo de Logistic regression usando el dataset balanceado.

Este predictor podrían mejorarse teniendo en cuento lo siguiente: 
* Eliminar los outliers del dataset debido a que en los boxplot de la Fig. 1 se observan varios de ellos.
* Probar con otro algoritmo de clasificación.

![fig/fig6.png](fig/fig6.png)

## Notebooks

El código de fuente para procesar el dataset, generar las gráficas y los modelos de casificación se encuentra en:

- [Análisis exploratorio y visualización efectiva](https://github.com/aguspesce/diabetes_predictor/blob/main/exploracion_visualizacion.ipynb)

- [Modelo de predicción de diabetes](https://github.com/aguspesce/diabetes_predictor/blob/main/modelo_predictor.ipynb)